In [ ]:
import pandas as pd
import numpy as np

import os
from kaggle.api.kaggle_api_extended import KaggleApi
import time

api = KaggleApi()
api.authenticate()

In [ ]:
competition = 'dm-ey-f-2024-segunda'
files_path = 'C:/Eugenio/Maestria/DMEyF/exp/competencia_2/'

In [ ]:
version = 'v002'

semillas = [122219, 109279, 400391, 401537, 999961]
puntos_corte = np.linspace(9000, 13000, 20).astype(int)

In [ ]:
# diccionario con la semilla y el corte de cada version
combinations = {}

counter = 1
for semilla in semillas:
    for corte in puntos_corte:
        combinations[counter] = {'seed':semilla,'corte':int(corte)}
        counter += 1

# path de cada version        
files = [f'{files_path}results_{version}_{i}.csv' for i in range(1,101)]

# descripcion de cada edicion
descriptions= [
    f"seed = {combinations[i]['seed']}, corte = {combinations[i]['corte']}"
    for i in range(1, len(combinations)+1)
]

In [ ]:
archivos = []
scores = []
descripciones = []

for i in range(len(files)):

    submitted = False
    while submitted == False:
        try:
            # Submit the file
            api.competition_submit(file_name=files[i],
                                   message=descriptions[i],
                                   competition=competition)
            time.sleep(10)
        except Exception as e:
            print(f'Error al subir {files[i]}: {e}')
            time.sleep(60)
        else:
            submitted = True 

            # Esperamos un poco para que kaggle no tire error
            time.sleep(10)

            submissions = api.competition_submissions(competition, page_size=1) # traigo solo la ultima
            ultima = submissions[0]
            
            archivo = ultima.fileName
            public_score = ultima.publicScore
            descripcion = ultima.description
            
            archivos.append(archivo)
            scores.append(public_score)
            descripciones.append(descripcion)
            
            print(f'{archivo} subido con exito, public score:{public_score}')

In [ ]:
resultados = pd.DataFrame({
    'Archivo': archivos,
    'Public score': scores,
    'Description':descripciones
})

df = pd.DataFrame({
    'seed': np.repeat(semillas, len(puntos_corte)),
    'corte': np.tile(puntos_corte, len(semillas))
})

resultados = pd.concat([resultados, df], axis=1)

In [ ]:
resultados.to_excel(f'{files_path}scores_{version}.xlsx', index=False)